In [2]:
import pickle
from crawl_parse_utils import check_url_relevance, detect
from rocksdict import Rdict
from tqdm import tqdm

In [3]:
db = Rdict('../../data/crawl_data')
titles_db = Rdict('../../data/titles')

In [8]:
# Print the length of the titles database
# also print some example titles and urls

count = 0
for key in titles_db.keys():
    count += 1
print('Number of titles in the database:', count)

count = 0
for key, value in titles_db.items():
    print(key, value)
    count += 1
    if count > 200:
        break

Number of titles in the database: 304182
Http://www.sacred-texts.com/tao Taoism - Sacred-Texts
http://1001genomes.org/data/MPI/MPISchneeberger2011/releases/current/Bur-0/hdr.annotation.TAIR8.txt 404 Not Found
http://1001genomes.org/data/MPI/MPISchneeberger2011/releases/current/Kro-0/del.annotation.TAIR8.txt 404 Not Found
http://1001genomes.org/data/MPI/MPISchneeberger2011/releases/current/Kro-0/snp.annotation.TAIR8.txt 404 Not Found
http://1001genomes.org/data/MPI/MPISchneeberger2011/releases/current/Ler-1/del.annotation.TAIR8.txt 404 Not Found
http://192brewing.com/ Home - 192 Brewing Company
http://2016.splashcon.org SPLASH 2016
http://2017.splashcon.org/track/splash-2017-OOPSLA SPLASH 2017 - OOPSLA - SPLASH 2017
http://2018.caaconference.org#content CAA 2018 Conference | CAA 2018
http://2018.caaconference.org/2017/07/14/call-for-workshops/#content Call for workshops | CAA 2018 Conference
http://2018.caaconference.org/2017/07/14/welcome-to-tubingen/#content Welcome to Tübingen | CAA 

In [6]:
total_items = 0
none_items = 0
visited_items = set()
rejected_items = set()

all_discovered_items = set()

for key, value in tqdm(db.items()):
    total_items += 1
    all_discovered_items.add(key)

    if value is None:
        none_items += 1
        rejected_items.add(key)
        continue

    if check_url_relevance(value) and detect(value.replace('\n', " "))['lang'] == 'en':
        visited_items.add(key)
    else:
        rejected_items.add(key)

print('Total items:', total_items)
print('Visited items:', len(visited_items))
print('Rejected items:', len(rejected_items))
print('All discovered items:', len(all_discovered_items))


0it [00:00, ?it/s]

2498277it [30:32, 1363.64it/s] 

Total items: 2498277
Visited items: 1068359
Rejected items: 1429918
All discovered items: 2498277


In [12]:
print("None items:", none_items)

None items: 161223


In [7]:
with open('../../data/crawl_state.pkl', 'rb') as f:
    cs = pickle.load(f)

In [13]:
for key in tqdm(all_discovered_items):
    if key not in cs['all_discovered_urls']:
        cs['all_discovered_urls'].add(key)

for key in tqdm(visited_items):
    if key not in cs['visited']:
        cs['visited'].add(key)

for key in tqdm(rejected_items):
    if key not in cs['rejected']:
        cs['rejected'].add(key)


100%|██████████| 1429918/1429918 [00:00<00:00, 1588567.42it/s]


In [17]:
cs['frontier'].extend(list(cs['to_visit']))
cs['to_visit'] = set()

In [18]:
with open('../../data/crawl_state.pkl', 'wb') as f:
    pickle.dump(cs, f)

In [5]:
crawled_urls = set()
title_urls = set()

for key in db.keys():
    crawled_urls.add(key)

for key in titles_db.keys():
    title_urls.add(key)

unavailable_urls = crawled_urls - title_urls

with open('../../data/unavailable_urls.pkl', 'wb') as f:
    pickle.dump(unavailable_urls, f)

In [6]:
print('Unavailable urls:', len(unavailable_urls))

Unavailable urls: 2511232
